In [14]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e7_p2_train2.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e7_p2_test2.csv')

display(train.head(2))
test.head(2)

print(train.shape)
print(train.head())
print(train.info())
print(train.describe())
print(train.isnull().sum())


,ID,연월,업종명,이용자구분,성별,이용자수,이용건수,이용금액
0,ID_5020,202201,여관업,법인,알수없음,7693,12105,3049021809
1,ID_5021,202201,여관업,제주도민,남성,3990,4291,294163241


(2245, 8)
        ID      연월          업종명 이용자구분    성별   이용자수   이용건수        이용금액
0  ID_5020  202201          여관업    법인  알수없음   7693  12105  3049021809
1  ID_5021  202201          여관업  제주도민    남성   3990   4291   294163241
2  ID_5022  202201          여관업  제주도민    여성   1495   1620   119277191
3  ID_5023  202201  전시 및 행사 대행업   내국인    남성  16299  18588   432933220
4  ID_5024  202201  전시 및 행사 대행업   내국인    여성  17350  19922   396316520
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      2245 non-null   object
 1   연월      2245 non-null   int64 
 2   업종명     2245 non-null   object
 3   이용자구분   2245 non-null   object
 4   성별      2245 non-null   object
 5   이용자수    2245 non-null   int64 
 6   이용건수    2245 non-null   int64 
 7   이용금액    2245 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 140.4+ KB
None
                  연월           이용자수           이

In [15]:
x = train.drop(['ID','이용금액'],axis=1)
y = train['이용금액']
target = test.drop('ID',axis=1)

In [16]:
mms_col = x.select_dtypes(exclude='object').columns
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x[mms_col] = mms.fit_transform(x[mms_col])
target[mms_col] = mms.transform(target[mms_col])

In [17]:
obj_col = x.select_dtypes('object').columns
for col in obj_col:
  print(col)
  print(x[col].unique())

업종명
['여관업' '전시 및 행사 대행업' '체인화 편의점' '피자, 햄버거, 샌드위치 및 유사 음식점업' '한식 음식점업' '호텔업'
 '화장품 및 방향제 소매업' '건강보조식품 소매업' '기타 대형 종합 소매업' '면세점' '비알콜 음료점업'
 '빵 및 과자류 소매업' '서양식 음식점업' '슈퍼마켓' '스포츠 및 레크레이션 용품 임대업']
이용자구분
['법인' '제주도민' '내국인' '중국' '기타외국' '동남아' '일본']
성별
['알수없음' '남성' '여성']


In [19]:
dum_col = ['이용자구분','성별']
x = pd.get_dummies(x, columns=dum_col,dtype=int)
target = pd.get_dummies(target, columns=dum_col, dtype=int)

In [13]:
print(x)

       연월 이용자구분    성별      이용자수      이용건수  업종명_건강보조식품 소매업  업종명_기타 대형 종합 소매업  \
0     0.0    법인  알수없음  0.017803  0.023309               0                 0   
1     0.0  제주도민    남성  0.009211  0.008238               0                 0   
2     0.0  제주도민    여성  0.003422  0.003086               0                 0   
3     0.0   내국인    남성  0.037770  0.035813               0                 0   
4     0.0   내국인    여성  0.040208  0.038386               0                 0   
...   ...   ...   ...       ...       ...             ...               ...   
2240  1.0   동남아  알수없음  0.000084  0.000091               0                 0   
2241  1.0    법인  알수없음  0.001086  0.000966               0                 0   
2242  1.0  제주도민    남성  0.003012  0.002681               0                 0   
2243  1.0  제주도민    여성  0.006012  0.005358               0                 0   
2244  1.0    중국  알수없음  0.000346  0.000357               0                 0   

      업종명_면세점  업종명_비알콜 음료점업  업종명_빵 및 과자류 소매업  업종명_서

In [20]:
le_col = ['업종명']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in le_col:
  x[col] = le.fit_transform(x[col])
  target[col] = le.transform(target[col])

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state=810)

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=810)
params = {'n_estimators':range(2,7),'max_depth':range(2,10)}
gscv = GridSearchCV(rf, param_grid=params, cv=5)
gscv.fit(x_train,y_train)
print(gscv.best_params_)

{'max_depth': 9, 'n_estimators': 6}


In [26]:
model = RandomForestRegressor(max_depth=9, n_estimators=6, random_state=810)
model.fit(x_train, y_train)
pred1 = model.predict(x_val)

In [32]:
from sklearn.metrics import mean_squared_error, r2_score
print(mean_squared_error(y_val, pred1, squared=False))
print(r2_score(y_val,pred1))

252303329.68691832
0.9844037006125709


In [33]:
result.to_csv('수험번호.csv',index=False)

NameError: name 'result' is not defined